# Pretrained model "model_whole.h5"

In [68]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow import keras 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import cv2
import random
import os
import json

import glob
from PIL import Image, ImageFile
import tensorflow as tf
import glob
import h5py
import tensorflow as tf
from tensorflow.python.tools import inspect_checkpoint as chkp
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow.contrib.slim as slim
# import tensorflow.contrib.slim.nets as nets
import h5py
ImageFile.LOAD_TRUNCATED_IMAGES = True


# !pip install tensorflow==1.15

In [69]:
tf.__version__

'1.15.0'

In [70]:
a="/nas/home/ellak-1000013/share/to_joul_2020-12-07/models-v02a3/model_whole.h5"
loaded =load_model("/nas/home/ellak-1000013/share/to_joul_2020-12-07/models-v02a3/model_whole.h5")


In [71]:
loaded.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [72]:
loaded = keras.models.Model(inputs=loaded.input, outputs=loaded.layers[-2].output)

In [73]:
# model = keras.Sequential()
# model.add(loaded)
# model.add(keras.layers.Dense(1024, activation='relu'))
# model.add(keras.layers.Dense(2, activation='softmax'))

In [74]:
# model.summary()

In [75]:
logdir = "/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/train_test_valid/train_test_valid/"
# " #This is the ABSOLUTE path to the database directory. Tensorboard and NIPGBoard
 
    #should be executed on this directory.
image_folder= "test" #Within the logdir, this folder contains the images to be parsed.

embedding_folder = "embedding" #Within the logdir, this folder will contain the embedding data.

video_folder = "video" #Within the logdir, this folder will contain the videos for the video player.

In [76]:

def formatImage(im):
    bgr = im.split()
    im = Image.merge("RGB", (bgr[2], bgr[1], bgr[0]))
    im = im.resize((256, 256), Image.ANTIALIAS)
    return np.asarray(im)



In [77]:
import random

def get_array(image_folder):
    png_files = glob.glob(os.path.join(logdir, image_folder, '*png'))
    jpg_files = glob.glob(os.path.join(logdir, image_folder, '*jpg'))
    sprite = os.path.join(logdir, image_folder, 'sprite.png')
    labels=[]
    if sprite in png_files:
        png_files.remove(sprite)
    image_files = sorted(png_files + jpg_files)
    X = np.zeros(shape=(len(image_files), 256, 256, 3), dtype=np.uint8)
    for i in range(len(image_files)):
        X[i] = formatImage(Image.open(image_files[i]))
        labels.append(os.path.basename(image_files[i]).split('_')[0])
    X = X.astype('float32')
    return X,labels,image_files

In [78]:
# X_train,x_train_label,image_files_train=get_array("train")

In [79]:
X_test,x_test_label,image_files_test=get_array("test")

In [80]:
# X_valid,x_valid_label,image_file_valid=get_array("valid")

In [81]:
# X_train

In [82]:
# label_encoder = LabelEncoder()
# x_train_label = label_encoder.fit_transform(x_train_label)

In [83]:
# x_train_label = pd.get_dummies(x_train_label)

In [84]:
# x_valid_label = pd.get_dummies(x_valid_label)

In [85]:
# x_valid_label.shape

In [86]:
# x_valid_label

In [87]:
# model.compile(optimizer="adam",
#               loss=keras.losses.BinaryCrossentropy(),
#               metrics=['accuracy'])

In [88]:
# model.fit(X_train,x_train_label,epochs=10,verbose=2,validation_data=(X_valid,x_valid_label))

In [89]:
# model = keras.models.Model(inputs=model.input, outputs=model.layers[-3].output)

In [90]:
intermediate_output = loaded.predict(X_test)

In [91]:
intermediate_output[0]

array([  0.     , 161.24004,  82.1569 , ..., 115.65387,   0.     ,
       210.59253], dtype=float32)

In [92]:
def create_sprite_image(img_paths, size):
    n_images = len(img_paths)
    img_h = size
    img_w = size
    n_plots = int(np.ceil(np.sqrt(n_images)))
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots, 3))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < n_images:
                img = cv2.imread(img_paths[this_filter])
                img = cv2.resize(img, (size,size))

                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = img
    
    return spriteimage

In [93]:
def makeProjector(logdir, embedding_data, embedding_name, embedding_folder, image_paths, labels=None):
    """data: np array"""
    sprite_size = 28
    sprite_path = os.path.join(logdir, embedding_folder, 'sprite.png')
    meta_path = os.path.join(logdir,embedding_folder,'metadata.tsv')
    checkpoint_path = os.path.join(logdir, embedding_folder, embedding_name+'.ckpt')

    embedding_var = tf.Variable(embedding_data, name=embedding_name)
    sess = tf.Session()
    sess.run(embedding_var.initializer)
    summary_writer = tf.summary.FileWriter(os.path.join(logdir,embedding_folder))
    summary_writer.add_graph(sess.graph)
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name

    embedding.metadata_path = meta_path
    embedding.sprite.image_path = sprite_path
    embedding.sprite.single_image_dim.extend([sprite_size, sprite_size])

    projector.visualize_embeddings(summary_writer, config)
    saver = tf.train.Saver([embedding_var])
    saver.save(sess, checkpoint_path, 1)

    if not os.path.isfile(sprite_path):
        sprite_image = create_sprite_image(image_paths, sprite_size)
        cv2.imwrite(sprite_path, sprite_image)

    # Create a metadata file if you want to using code similar to this:
    if not os.path.isfile(meta_path):
        if labels == None:
            with open(meta_path,'w') as f:
                f.write("Index\tFilename\t_sync_id\n")
                for index, img_path in enumerate(image_paths):
                    f.write("%d\t%s\t%d\n" % (index, img_path.split('/')[-1].split('\\')[-1], index))
        else:    
            with open(meta_path,'w') as f:
                f.write("Index\tLabel\tFilename\t_sync_id\n")
                
                for index,(label, img_path) in enumerate(zip(labels, image_paths)):
                    f.write("%d\t%s\t%s\t%d\n" % (index,label, img_path, index))

In [94]:
makeProjector(logdir=logdir,
    embedding_data=intermediate_output,
    embedding_name='dense_1',        
    embedding_folder=embedding_folder,
    image_paths=image_files_test)

In [95]:
config_dict = {
    "default": {
        "embedding_folder": embedding_folder,
        "name": "DENSENET",
        "image_folder": image_folder,
        "video_folder": video_folder
    },
    
    "training": {
        "algorithm_path": "/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/algorithms",
        "embedding_folder": "densenet",
        "name": "DENSENET+KIRA",
        "algorithm": {
            "callable": "run",
            "keyword_arguments": {
                "epoch": "1"
            },
            "arguments": [],
            "file": "densenet"
        }
    }
}
with open(os.path.join(logdir, 'cnf.json'), 'w') as fp:
    json.dump(config_dict, fp)

In [96]:
plugins_dict = {
    "plugins": ["executer", "selected", "image", "projector", "video"]
}
with open(os.path.join(logdir, 'plugins.json'), 'w') as fp:
    json.dump(plugins_dict, fp)

In [97]:
with open(os.path.join(logdir,embedding_folder,'metadata.tsv'), 'wt') as metadata_file:
    metadata_file.write('Index\tLabel\tFilename\t_sync_id\n')
    for i in range(len(image_files_test)):
        metadata_file.write('%d\t%s\t%s\t%d\n' % (i, image_files_test[i].split('/')[-1].split('\\')[-1].split('_')[0], image_files_test[i].split('/')[-1].split('\\')[-1], i))
    metadata_file.close()